In [38]:
import pandas 

Retrieving business, training and testing reviews dataframe from pickle

In [39]:
business_df = pandas.read_pickle('business.pkl')
review_df = pandas.read_pickle('reviews_filtered.pkl')

In [59]:
from sklearn.model_selection import KFold 
kf = KFold(n_splits=4) 
kf.get_n_splits(review_df)  
print(kf)   
KFold(n_splits=4, random_state=None, shuffle=False)
results = []
for train_index, test_index in kf.split(review_df): 
    results.append([train_index,test_index])
train_index, test_index = results[3]
review_train_df, review_test_df = review_df.iloc[train_index], review_df.iloc[test_index]

KFold(n_splits=4, random_state=None, shuffle=False)


Calculating overall rating average and average of ratings by a particular user and for a particular business

In [60]:
overall_avg_rating = review_train_df["stars"].mean() #average of all the ratings in the dataset
avg_user_rating_df = review_train_df.groupby(["user_id"]).mean() #average of all the ratings by a particular user
avg_business_rating_df = review_train_df.groupby(["business_id"]).mean() #average of all the ratings for a particular business

Calculating rating by a particular user for a particular business by baseline approach

In [61]:
def calc_rating_baseline(user_id, business_id):
    user_bias = 0
    business_bias = 0
    user_rating = avg_user_rating_df[avg_user_rating_df.index == user_id]["stars"]
    if not user_rating.empty:
        user_bias = user_rating.item() - overall_avg_rating
    business_rating = avg_business_rating_df[avg_business_rating_df.index == business_id]["stars"]
    if not business_rating.empty:
        business_bias = business_rating.item() - overall_avg_rating
    return overall_avg_rating + user_bias + business_bias

Calculating user to business ratings for the held out test dataset

In [62]:
review_test_result_df = review_test_df.copy(deep = True)
review_test_result_df["stars"] = 0

In [63]:
review_test_result_df['stars'] = review_test_df.apply(lambda x: calc_rating_baseline(x[0], x[1]),axis = 1)

Save baseline results to pickle

In [64]:
review_test_result_df.to_pickle("./baseline_results.pkl")

Calculating RMSE for the baseline results

In [71]:
def calc_RMSE():
    return ((review_test_result_df.stars - review_test_df.stars) ** 2).mean() ** 0.5
result = calc_RMSE()
result

1.08422565507338